In [ ]:
import pandas as pd
import cv2
import numpy as np
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
mp_drawing_styles = mp.solutions.drawing_styles
from utils import *

In [ ]:
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val),
                  'z{}'.format(val), 'v{}'.format(val)]

In [ ]:
play_video_from_file('./videos/oneFoot.avi')

In [ ]:
apply_mediapipe_holistic_model('./videos/oneFoot.avi')

In [ ]:
def standing_on_one_leg_grader(hold_time,fall_flag,other_person_flag):
    if hold_time > 10:
        return 4
    elif 5 <= hold_time <= 10:
        return 3
    elif 3 <= hold_time < 5:
        return 2
    elif hold_time < 3 and not fall_flag:
        return 1
    elif other_person_flag:
        return 0


In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
distance_band = 0.1
cap = cv2.VideoCapture('./oneStepSide.avi')
captured_video_fps = cap.get(cv2.CAP_PROP_FPS)
frame_idx = 0
start_time = cv2.getTickCount()

#######GLOBALS##############
history_array = []
leg_raise_begin_time = None
leg_raise_end_time = None
tries = 0
#########FLAGS##############
fall_flag = False
other_person_flag = False
is_right_leg_lifted = False
is_left_leg_lifted = False
first_time = False
############################
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    frame_idx+=1
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    
    try:
      ######################################################
      #Extract Joint coordinates
      my_landmarks = results.pose_landmarks.landmark
      joint_coordinates = get_coordinates(my_landmarks)
      right_heel = [my_landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x , my_landmarks[mp_pose.PoseLandmark.RIGHT_HEEL].y, my_landmarks[mp_pose.PoseLandmark.RIGHT_HEEL].z]
      left_heel = [my_landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x , my_landmarks[mp_pose.PoseLandmark.LEFT_HEEL].y, my_landmarks[mp_pose.PoseLandmark.LEFT_HEEL].z]
      left_ankle = joint_coordinates[6][2]
      right_ankle = joint_coordinates[7][2]

      # print("right heel are: ",right_heel," left heel are: ",left_heel)
      
      if right_ankle[1] - left_ankle[1] > distance_band:
        print("left leg is lifted.")
        is_left_leg_lifted = True

      if left_ankle[1] - right_ankle[1] > distance_band:
        print("right leg is lifted.")
        is_right_leg_lifted = True
      #############################

      #############################
      end_time = cv2.getTickCount()   
      total_time = (end_time - start_time) / cv2.getTickFrequency()
      # print("The fps is: ",frame_idx/total_time)
      multiply_factor = 20/(frame_idx/total_time)
      real_time = total_time/multiply_factor
      cv2.rectangle(image, (0, 0), (640,60), (255, 0, 0), -1)
      cv2.putText(image,str(real_time), (50, 50), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
      #################################################################
      
      if first_time:
        now = real_time
        time = now - leg_raise_begin_time
        if time >=10:
          print("The result for this exersize is: ",standing_on_one_leg_grader(time,fall_flag,other_person_flag))
          break

      if is_right_leg_lifted and not first_time:
        leg_raise_begin_time = real_time
        first_time = True
        history_array.append('raised right')


      if not is_right_leg_lifted and first_time:
        leg_raise_end_time = real_time
        first_time = False
        history_array.append('normal right')

      
      if is_left_leg_lifted and not first_time:
        leg_raise_begin_time = real_time
        first_time = True
        history_array.append('raised left')
      
      if not is_left_leg_lifted and first_time:
        leg_raise_end_time = real_time
        first_time = False
        history_array.append('normal left')

      
      if leg_raise_end_time and leg_raise_begin_time:
        time = leg_raise_end_time - leg_raise_begin_time
        # print("The result for this exesize is: ",standing_on_one_leg_grader(time,fall_flag,other_person_flag))
        result = standing_on_one_leg_grader(time,fall_flag,other_person_flag)
        if result == 3:
          print("The result for this exersize is: ",result)
          break
        elif result == 2:
          print("The result for this exersize is: ",result)
          break
        elif result == 1:
          print("The result for this exercize is: ",result)
          break
        elif result ==0:
          tries +=1
          if tries >= 3:
            print("The result for this exersize is: ",result)
            break
      #################################################################
    except Exception as e:
        print('error',e)
        pass

    # Flip the image horizontally for a selfie-view display.
    flipped_frame = cv2.flip(image, 0)

    cv2.imshow('MediaPipe Holistic',image)
    is_right_leg_lifted = False
    is_left_leg_lifted = False
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()